# OMS

In [3]:
# -*- coding: utf-8 -*-
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from urllib.parse import urljoin

In [4]:
import requests
from bs4 import BeautifulSoup

url = "https://www.afro.who.int/fr/news/feature_stories"
html = requests.get(url)

soup = BeautifulSoup(html.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="fr" prefix="og: https://ogp.me/ns#">
 <head>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
  <meta content="Africa, health, malaria, infectious, medicine, disease, infection, WHO, Polio, Aids, Organization, Organization, country, Africa, African, world, office, region, world health organization, world health organisation" name="keywords">
   <meta content="The World Health Organization (WHO) is building a better future for people everywhere. Health lays the foundation for vibrant and productive communities, stronger economies, safer nations and a better world. Our work touches lives around the world every day – often in invisible ways. As the lead health authority within the United Nations (UN) system, we help ensure the safety of the air we breathe, the food we eat, the water we drink and the medicines and vaccines that treat and protect us. The Organization aims to provide every child, 

In [7]:


# ---------------------------
# CONFIG
# ---------------------------
URL = "https://www.afro.who.int/fr/news/feature_stories"  # ou "file:///path/to/html-oms.md"
MAX_ARTICLES = 60
MAX_PAGES = 10        # sécurité : ne pas boucler indéfiniment
WAIT_TIMEOUT = 12

# ---------------------------
# Options & driver (Firefox)
# ---------------------------
options = Options()
options.headless = True
# quelques préférences utiles (désactiver images accélère parfois)
options.set_preference("permissions.default.image", 2)
options.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", "false")
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)
wait = WebDriverWait(driver, WAIT_TIMEOUT)

articles = []
seen_links = set()

try:
    print("Ouverture de la page :", URL)
    driver.get(URL)
    time.sleep(2)

    # attendre que la zone principale soit présente
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.view-content")))

    page_index = 0
    while len(articles) < MAX_ARTICLES and page_index < MAX_PAGES:
        print(f"\n--- Page {page_index+1} ---")
        # assurer que les tuiles d'articles sont chargées
        time.sleep(1)
        rows = driver.find_elements(By.CSS_SELECTOR, "div.view-content > div.row.views-row, article.news")
        # fallback : chercher tous les article nodes
        if not rows:
            rows = driver.find_elements(By.CSS_SELECTOR, "article.news")

        print(f"Items trouvés sur la page: {len(rows)}")

        for row in rows:
            if len(articles) >= MAX_ARTICLES:
                break
            try:
                # Titre et lien
                a = None
                try:
                    a = row.find_element(By.CSS_SELECTOR, "h3.teaser-full__title a, .node-title a, .views-field-title a")
                except:
                    # fallback find any link inside row
                    anchors = row.find_elements(By.TAG_NAME, "a")
                    a = anchors[0] if anchors else None

                if not a:
                    continue
                href = a.get_attribute("href") or a.get_attribute("data-href") or ""
                # normaliser lien relatif
                href = urljoin("https://www.afro.who.int", href)

                if href in seen_links:
                    continue
                seen_links.add(href)

                title = a.text.strip()
                if not title:
                    # parfois titre est dans child span
                    title = a.get_attribute("title") or "Titre non disponible"

                # date
                date_text = ""
                try:
                    time_el = row.find_element(By.CSS_SELECTOR, "time, .field--name-field-date time")
                    date_text = time_el.get_attribute("datetime") or time_el.text.strip()
                except:
                    date_text = ""

                # extrait / teaser
                teaser = ""
                try:
                    teaser_el = row.find_element(By.CSS_SELECTOR, ".content p, .field--name-body p")
                    teaser = teaser_el.text.strip()
                except:
                    teaser = ""

                print(f"Récupère article {len(articles)+1}: {title[:80]}")

                # Ouvrir article dans un nouvel onglet et récupérer le texte complet
                article_text = ""
                try:
                    driver.execute_script("window.open(arguments[0], '_blank');", href)
                    driver.switch_to.window(driver.window_handles[-1])
                    # attendre le contenu principal de l'article
                    try:
                        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main, article, .content, .field--name-body")))
                    except:
                        pass
                    time.sleep(1)
                    # tenter plusieurs sélecteurs pour le corps de l'article
                    selectors = ["article.node", "main", ".field--name-body", ".content", ".post-content", ".node--type-news"]
                    body_text = ""
                    for sel in selectors:
                        try:
                            el = driver.find_element(By.CSS_SELECTOR, sel)
                            body_text = el.text.strip()
                            if body_text:
                                break
                        except:
                            body_text = ""
                    article_text = body_text or ""
                except Exception as e:
                    print("  Erreur ouverture article:", e)
                    article_text = ""
                finally:
                    # fermer onglet article et revenir
                    if len(driver.window_handles) > 1:
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])

                articles.append({
                    "source": "WHO AFRO",
                    "titre": title,
                    "date": date_text,
                    "lien": href,
                    "teaser": teaser,
                    "texte": article_text
                })
                print("  ✓ ajouté")
            except Exception as e:
                print("  ⚠ erreur item:", e)
                continue

        # pagination : aller à la page suivante si nécessaire
        if len(articles) >= MAX_ARTICLES:
            break

        page_index += 1
        try:
            # trouver le lien "next" dans la pagination
            next_el = driver.find_element(By.CSS_SELECTOR, "ul.pager__items a[rel='next'], .pager__item a[rel='next']")
            next_href = next_el.get_attribute("href")
            if not next_href:
                # si pas de href, break
                print("Pas de lien 'next' trouvé -> arrêt pagination")
                break
            print("Navigation vers la page suivante...")
            driver.get(next_href)
            # courte pause pour chargement
            time.sleep(1.2)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.view-content")))
        except Exception as e:
            print("Fin de la pagination ou erreur:", e)
            break

    # Fin boucle pages

    # Construire DataFrame
    df = pd.DataFrame(articles)
    print(f"\nTotal articles récupérés: {len(df)}")

    if not df.empty:
        csv_path = "../outputs/oms_articles.csv"
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        #df.to_excel(xlsx_path, index=False, engine="openpyxl")
        print("Données sauvegardées :", csv_path)
    else:
        print("Aucun article récupéré.")

except Exception as e_main:
    print("Erreur générale:", e_main)

finally:
    try:
        driver.quit()
    except:
        pass

# fin du script


Ouverture de la page : https://www.afro.who.int/fr/news/feature_stories

--- Page 1 ---
Items trouvés sur la page: 20
Récupère article 1: Soins, compassion et guérison du plus jeune patient de l’épidémie d’Ebola en Rép
  ✓ ajouté
Récupère article 2: Comprendre la prévalence, les risques et les mécanismes de prévention du cancer 
  ✓ ajouté
Récupère article 3: Les centres de prise en charge au cœur de la lutte contre la mpox au Burundi
  ✓ ajouté
Récupère article 4: L’épidémie de choléra en recul au Congo
  ✓ ajouté
Récupère article 5: RDC : le deuil coutumier de 40 jours aide à freiner la transmission d’Ebola
  ✓ ajouté
Récupère article 6: Cameroun : un espoir pour la santé des adolescents et jeunes séropositifs
  ✓ ajouté
Récupère article 7: RDC : une source d’eau qui transforme la vie à Bulape, au-delà d’Ebola
  ✓ ajouté
Récupère article 8: Ebola : le Congo en alerte pour contenir le risque
  ✓ ajouté
Récupère article 9: Ouganda : soutien psychosocial et soins de santé mentale pour l